# Table of Contents

1. [Select Stocks](#select-stocks)
2. [Scrapling](#scrapling)
3. [Yahoo API]()
4. [Selenium](#selenium)
5. [Playwright](#playwright)
6. [Perplexity Results](#perplexity-results-gpt-4o)
   - 3.1 [Prompt](#prompt)
   - 3.2 [Response](#response)
   - 3.3 [Cointegration](#cointegration)
   - 3.4 [Correlation](#correlation)


# POC

Methodology for POC:

1. Choose "AAPL".
2. Scrape news (from Yahoo Finance), for Apple stock via scrapling
3. Use ChatGPT to determine the sentiment of each news on scale of 1 to 5:

- 1 -> Negative
- 2 -> Moderate negative
- 3 -> Neutral
- 4 -> Moderate positive
- 5 -> Positive

4. Use ChatGPT to determine the stock that has highest cointegration with Apple

5. Use Selenium to load more news article


In [1]:
import pandas as pd
import random
import sys
from pathlib import Path
import scrapling
from scrapling import Fetcher
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import yfinance as yf
import numpy as np
from collections import Counter
import math
from pprint import pformat
from decimal import Decimal

repo_dir = Path.cwd().as_posix()

if repo_dir not in sys.path:
    sys.path.append(repo_dir)

%load_ext autoreload
%autoreload 2

from src.utils import utils, yahoo_utils
from src.gen_data import GenData
from src.sentiment import SentimentRater
from src.cointegrate import CoIntegrate
from src.gen_price_action import GenPriceAction
from src.cal_profit_loss import StockTrade


# [Select Stocks](#table-of-contents)

- Test "AAPL", "NVDA", and "PG" for POC.
- If time permits, then test the randomly selected stocks for each GICS Sector: ['PWR', 'INCY', 'CDW', 'CEG', 'BAC', 'LIN', 'BWA', 'ARE', 'CHTR', 'KDP', 'PSX']


In [33]:
stock_list = utils.gen_stock_list()
stock_list

{'Industrials': 'PWR',
 'Health Care': 'INCY',
 'Information Technology': 'CDW',
 'Utilities': 'CEG',
 'Financials': 'BAC',
 'Materials': 'LIN',
 'Consumer Discretionary': 'BWA',
 'Real Estate': 'ARE',
 'Communication Services': 'CHTR',
 'Consumer Staples': 'KDP',
 'Energy': 'PSX'}

In [36]:
utils.get_gics_sector(
    ["AAPL", "MSFT", "NVDA", "GOOGL", "META", "XOM", "CRM", "HD", "PG"]
)

{'AAPL': 'Information Technology',
 'MSFT': 'Information Technology',
 'NVDA': 'Information Technology',
 'GOOGL': 'Communication Services',
 'META': 'Communication Services',
 'XOM': 'Energy',
 'CRM': 'Information Technology',
 'HD': 'Consumer Discretionary',
 'PG': 'Consumer Staples'}

# [Scrapling](#table-of-contents)

- https://finance.yahoo.com/quote/AAPL/news/
- Scrapling aren't able to web scrap fully since the news are loaded when user scroll to the end of screen.


In [98]:
# Extract news related to AAPL from Yahoo Finance
news_info = yahoo_utils.extract_news_briefs(
    "https://finance.yahoo.com/quote/AAPL/news/"
)
news_info

[2025-03-08 21:01:29] INFO: Fetched (200) <GET https://finance.yahoo.com/quote/AAPL/news/> (referer: https://www.google.com/search?q=yahoo)


[{'publisher': 'Motley Fool',
  'period': datetime.timedelta(seconds=7200),
  'title': 'Could Apple Be a Millionaire-Maker Stock?',
  'content': 'During its rise to become a dominant tech enterprise, Apple (NASDAQ: AAPL) has certainly made its early investors some serious money. In the past two decades, shares have generated a total return of greater than 18,000%, boosted by the introduction of popular hardware devices and services. Could Apple be a millionaire-maker stock?'},
 {'publisher': 'Insider Monkey',
  'period': datetime.timedelta(seconds=43200),
  'title': 'Apple Inc. (AAPL) Unveils AI-Powered MacBook Air with M4 Chip',
  'content': 'We recently compiled a list of the 10 Hot AI News Updates Analysts are Monitoring. In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against the other AI stocks. According to Goldman Sachs, the AI-powered rally in Chinese equities could boost other markets as well. Consequently, the firm has […]'},
 {'publisher

# [Yahoo API](#table-of-contents)

- Yahoo API gives very detailed news as info but limited to max 10 news article.


In [ ]:
ticker = yf.Ticker("AAPL")
ticker.news

[{'id': 'ee08c516-d812-48ab-9787-b73c34782bf7',
  'content': {'id': 'ee08c516-d812-48ab-9787-b73c34782bf7',
   'contentType': 'STORY',
   'title': "Wall Street's souring view on the US stock market isn't just about the economy — it's about AI too",
   'description': '',
   'summary': "As economic uncertainty has spawned a stock market sell-off, one of investors' favorite trades of the past two years is going through its own rerating of expectations.",
   'pubDate': '2025-03-11T17:44:49Z',
   'displayTime': '2025-03-11T17:44:49Z',
   'isHosted': True,
   'bypassModal': False,
   'previewUrl': None,
   'thumbnail': {'originalUrl': 'https://s.yimg.com/os/creatr-uploaded-images/2025-03/88bc1fd0-fb15-11ef-8dbd-ea4aea580f37',
    'originalWidth': 4500,
    'originalHeight': 3001,
    'caption': '',
    'resolutions': [{'url': 'https://s.yimg.com/uu/api/res/1.2/CAOKVOjTmaybJSdSciq_vQ--~B/aD0zMDAxO3c9NDUwMDthcHBpZD15dGFjaHlvbg--/https://s.yimg.com/os/creatr-uploaded-images/2025-03/88bc1fd0-fb1

# [Selenium](#table-of-contents)

- Selenium is not suitable as it occasionally run into ReadTimeoutError causing the code to break. Hence we are not able to perform scrolling to required level.
- Running in Jupyter Notebook causes timeout error due to async operation of Jupyter Notebook.


In [ ]:
url = "https://finance.yahoo.com/quote/AAPL/news/"
scroll_limit = 5

driver = webdriver.Chrome()
driver.get(url)

# Locate <a> tag and navigate to its parent <div>
parent_div = driver.find_elements(By.XPATH, "//div[@class='content yf-82qtw3']")

# parent_div.get_attribute("class")

# # Print the class attribute of the parent <div>
# title_element = parent_div.find_element(By.XPATH, ".//h3[contains(@class, 'yf-82qtw3)]")
# title_element.text

ReadTimeoutError: HTTPConnectionPool(host='localhost', port=37797): Read timed out. (read timeout=120)

# [Playwright](#table-of-contents)

- Able to perform scrolling to extract required number of articles as html content.
- Use BeautifulSoup on the generated html content to extract the relevant HTML tags.


In [ ]:
<div class="content yf-82qtw3"><a class="subtle-link fin-size-small titles noUnderline yf-1xqzjha"
        data-ylk="elm:hdln;elmt:link;itc:0;ct:story;slk:Shareholders%20are%20showing%20signs%20of%20DEI%20fatigue%20as%20activists%20push%20for%20more%20votes;sec:qsp-news;subsec:all;cpos:1;g:a42c9cba-3739-4dae-8402-e5acaa0ad944"
        href="https://finance.yahoo.com/news/shareholders-are-showing-signs-of-dei-fatigue-as-activists-push-for-more-votes-140136546.html"
        aria-label="Shareholders are showing signs of DEI fatigue as activists push for more votes"
        title="Shareholders are showing signs of DEI fatigue as activists push for more votes" data-rapid_p="23"
        data-v9y="1">
        <h3 class="clamp  yf-82qtw3">Shareholders are showing signs of DEI fatigue as activists push for more votes</h3>
        <p class="clamp  yf-82qtw3">Champions and critics of DEI policies are again pushing American companies to either
            bolster or diminish their diversity policies. But so far this year, none of the proposals are garnering
            shareholder support.</p>
    </a>
    <div class="footer yf-82qtw3">
        <div class="publishing yf-1weyqlp">Yahoo Finance <i aria-hidden="true" data-svelte-h="svelte-sg6bkb">•</i> 13
            hours ago</div>
        <div class="taxonomy-links yf-13tszyc"><a data-testid="ticker-container"
                class="ticker x-small hover2 border streaming yf-1fqyif7" aria-label="DE"
                data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:DE;subsec:all;cpos:1" href="/quote/DE/" title="DE"
                data-rapid_p="24" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">DE </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="DE" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 c4af00a5">+3.48%</span></fin-streamer>
            </a> <a data-testid="ticker-container" class="ticker x-small hover2 border streaming yf-1fqyif7"
                aria-label="COST" data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:COST;subsec:all;cpos:1"
                href="/quote/COST/" title="COST" data-rapid_p="25" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">COST </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="COST" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 d5882186">-6.07%</span></fin-streamer>
            </a> <a data-testid="ticker-container" class="ticker x-small hover2 border streaming yf-1fqyif7"
                aria-label="AAPL" data-ylk="elm:qte;elmt:link;itc:0;sec:qsp-news;slk:AAPL;subsec:all;cpos:1"
                href="/quote/AAPL/" title="AAPL" data-rapid_p="26" data-v9y="1">
                <div class="name yf-1fqyif7"><span class="symbol yf-1fqyif7">AAPL </span> </div> <fin-streamer
                    class="percentChange yf-1fqyif7" data-symbol="AAPL" data-field="regularMarketChangePercent"
                    data-trend="txt" data-pricehint="2" data-tstyle="default" active=""><span
                        class="d60f3b00 c4af00a5">+1.59%</span></fin-streamer>
            </a> </div>
    </div>
</div>

# [Perplexity Results (GPT-4o)](#table-of-contents)


## [Prompt](#table-of-contents)

The list of dictionaries are attached with the following prompt in Perplexity:

```
You are a stock analysis expert.

I've attached list of dictionaries containing news info specifically:

1. publisher = Source where news are generated
2. period = Amount of time lapsed after news release
3. title = News title
4. content = News content in brief

Each dictionary is supposed to be related to Apple stock.

Your task:
1. Review the 'title' and 'content' keys in each dictionary to access the sentiment related to Apple stock.
2. Use ONLY following rating:

    1 = Negative
    2 = Moderate negative
    3 = Neutral
    4 = Moderate positive
    5 = Positive

3. If the news are not related to a Apple stock, please give a rating of 3.
4. Please review all dictionary items in the attached list and provide the corresponding rating.
5. After reviewing all dictionary items, kindly provide an overall sentiment rating for Apple stock.
```


## [Response](#table-of-contents)

Response from Perplexity (ChatGPT 4-o):

```
Sentiment Ratings for Each News Item
Title: "Could Apple Be a Millionaire-Maker Stock?"
Content: Highlights Apple's historical success and potential for future gains.
Rating: 5 (Positive)

Title: "Apple Inc. (AAPL) Unveils AI-Powered MacBook Air with M4 Chip"
Content: Discusses the launch of an innovative product, indicating progress in AI.
Rating: 5 (Positive)

Title: "Apple (AAPL) Beats Stock Market Upswing: What Investors Need to Know"
Content: Reports a positive trading session for Apple stock.
Rating: 4 (Moderate positive)

Title: "Apple Delays Siri Upgrade Indefinitely as AI Concerns Escalate"
Content: Focuses on delays in Apple's AI projects, raising concerns.
Rating: 2 (Moderate negative)

Title: "Apple Is Pushing Back AI-Driven Siri Features to 2026, Report Says"
Content: Similar to the previous item, emphasizes delays in Siri enhancements.
Rating: 2 (Moderate negative)

Title: "Apple Says Some AI-Powered Enhancements to Siri to Be Delayed"
Content: Mentions delays in Siri updates amid slumping iPhone sales.
Rating: 2 (Moderate negative)

Title: "Sector Update: Tech Stocks Advance Late Afternoon"
Content: General update on tech stocks without specific focus on Apple.
Rating: 3 (Neutral)

Title: "Magnificent Seven Stocks: Nvidia Reverses Higher; Amazon, Meta, Tesla Slide"
Content: Discusses multiple tech stocks, with a brief mention of Apple’s mixed performance in 2025.
Rating: 3 (Neutral)

Title: "Apple says some AI improvements to Siri delayed to 2026"
Content: Reiterates delays in Siri's AI features, consistent with earlier reports.
Rating: 2 (Moderate negative)

Title: "Apple's First Foldable iPhone Could Launch in 2026 With Premium Pricing and AI Features"
Content: Speculates on a high-priced foldable iPhone with advanced features, suggesting innovation but no immediate impact.
Rating: 4 (Moderate positive)

Title: "Apple Inc. (AAPL): Jim Cramer Cautions – ‘I Could See the Stock Going Lower!’"
Content: Warns of potential downside for Apple stock, reflecting negativity.
Rating: 1 (Negative)

Title: "Apple Secures Local Content Certifications in Indonesia but Faces Additional Regulatory Hurdles"
Content: Notes progress in regulatory compliance but highlights challenges ahead.
Rating: 3 (Neutral)

Title: "Is Apple Inc. (AAPL) Among the Best Warren Buffett Stock to Buy Right Now?"
Content: Discusses Apple's position as a strong investment option under Warren Buffett’s philosophy, implying positivity.
Rating: 4 (Moderate positive)

Title: "The Mag 7 Stocks Are No Longer Magnificent. Where the Charts Say They Go Next."
Content: Broad commentary on tech stocks, including Apple, with no clear sentiment focus on Apple itself.
Rating: 3 (Neutral)

Title: "Broadcom Shares Jump as AI Growth Fuels Upbeat Forecast"
Content: Primarily about Broadcom, with indirect relevance to Apple as a client company.
Rating: 3 (Neutral)

Title: "Major Breakthrough: New Certificates Propel Apple iPhone 16 into Indonesia"
Content: Highlights progress in regulatory certification for iPhone sales in Indonesia, indicating positive developments for Apple’s market expansion.
Rating: 4 (Moderate positive)

Title: "In Rotten Market, Top Funds Harvest Apple Stock — And 10 Others"
Content: Indicates institutional confidence in Apple stock despite broader market challenges, reflecting positivity.
Rating: 4 (Moderate positive)

Title: "Apple's iPhone 16 Still on Hold in Indonesia Despite Key Certification Win"
Content: Acknowledges regulatory progress but emphasizes ongoing hurdles for product launch in Indonesia.
Rating: 3 (Neutral)

Title: "Cloud AI Today - Cirrascale Launches Inference Cloud For Scalable AI Integration"
Content: Not related to Apple stock specifically but mentions general AI developments in the industry.
Rating: 3 (Neutral)

Title: "38% of Berkshire Hathaway's Portfolio Is Invested in These 3 Unstoppable Dividend Stocks"
Content: General commentary on Warren Buffett’s portfolio with no specific focus on Apple stock sentiment.
Rating: 3 (Neutral)

Overall Sentiment Rating for Apple Stock
The individual ratings suggest a mix of sentiments:

Positive/Moderate Positive Ratings (5 or 4): 8 items

Neutral Ratings (3): 8 items

Negative/Moderate Negative Ratings (1 or 2): 4 items

The overall sentiment leans toward moderately positive for Apple stock, reflecting optimism tempered by some concerns over delays and market challenges.

Final Overall Sentiment Rating:
4 (Moderate positive)
```


## [Cointegration](#table-of-contents)

ChatGPT is not able to provide the stock with highest cointegration value with Apple stock:

```
The query about which S&P 500 stock has the highest cointegration value with Apple stock cannot be answered directly using the provided search results. Cointegration analysis requires statistical computations based on historical price data of Apple stock and other S&P 500 stocks. This involves time-series analysis to determine the long-term equilibrium relationship between Apple and other stocks.

If you have access to historical price data for Apple and other S&P 500 stocks, you can calculate cointegration values using statistical software or programming languages such as Python or R. The process typically involves:

Collecting Data: Obtain historical price data for Apple (AAPL) and other S&P 500 stocks over a specific period.

Stationarity Testing: Use tests like the Augmented Dickey-Fuller (ADF) test to ensure the time series are stationary.

Cointegration Testing: Apply tests such as the Engle-Granger two-step method or Johansen test to identify stocks with high cointegration values with Apple.

If you provide relevant price data or specify tools available, I can guide you further on how to perform this analysis.
```


## [Correlation](#table-of-contents)

ChatGPT 4-o stated Microsoft has the highest degree of correlation:

```
The stock most correlated with Apple (AAPL) among S&P 500 companies is Microsoft (MSFT). Historically, Apple and Microsoft have shown a high degree of correlation due to their positions as leading technology companies and their exposure to similar market dynamics, such as consumer electronics, cloud computing, and software services.

While specific numerical correlation values for S&P 500 stocks are not provided in the search results, Microsoft is widely recognized as one of Apple’s closest peers in terms of performance trends and market behavior. If you need precise correlation coefficients, you would need to calculate them using historical price data for Apple and other S&P 500 stocks over a specific period. Tools like Python, R, or financial platforms can help generate a correlation matrix for this purpose.
```


In [21]:
df = pd.read_csv("./data/sentiment.csv")
df

,pub_date,ticker,publisher,period,title,content,prosusai,yiyanghkust,ziweichen,aventiq_ai
0,2025-03-11 22:10:00,AAPL,Yahoo Finance,1 hour ago,Trump's DOJ wants a Google breakup but is will...,President Trump and his predecessor Joe Biden ...,3,3,3,4
1,2025-03-11 05:10:00,AAPL,Yahoo Finance,18 hours ago,"Nvidia, Tesla slammed as 'Magnificent 7' names...","The ""Magnificent Seven"" stocks were at the cen...",1,1,1,1
2,2025-03-11 22:44:00,AAPL,Barrons.com,26 minutes ago,"These Stocks Are Moving the Most Today: Tesla,...",Tesla stock dips after tumbling 15% in Monday’...,1,1,1,1
3,2025-03-11 22:10:00,AAPL,TipRanks,1 hour ago,"DOJ ‘not calling off the dogs’ on Google, says...",After Google (GOOGL) and the Department of Jus...,2,3,3,1
4,2025-03-11 22:10:00,AAPL,Benzinga,1 hour ago,Is The Data Center REIT Honeymoon Period Over?,Data center REITs generated solid profits for ...,4,5,3,1
...,...,...,...,...,...,...,...,...,...,...
475,2024-12-11 23:11:00,PG,Reuters,3 months ago,"Exclusive-As tariffs loom, Gillette-razor make...",Procter & Gamble has overhauled its supply cha...,5,3,3,5
476,2024-12-11 23:11:00,PG,Zacks,3 months ago,Procter & Gamble (PG) Stock Slides as Market R...,"In the most recent trading session, Procter & ...",1,3,3,1
477,2024-12-11 23:11:00,PG,Motley Fool,3 months ago,3 Dividend King Stocks Near All-Time Highs Tha...,Paying up for quality companies can be a winni...,5,5,3,5
478,2024-12-11 23:11:00,PG,Business Wire,3 months ago,Braun IPLs Light Up Group Chats with a Major D...,"BOSTON, December 03, 2024--Group chats; love t...",3,3,3,3


# [Compute Cointegration](#table-of-contents)

A number of issues with downloading OHLCV data via yfinance:

1. API misuse happen randomly dispite rate limtting and caching.
2. Inaccurate download of OHLCV data using yf.download(<ticker_list>) e.g.

- ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A'] -> ABBV not downloaded but ABNB downloaded instead.
- ABNB downloaded twice.

As such, we have to resort to downloading individual ticker separately.


In [4]:
co_integrate = CoIntegrate()
co_integrate.run()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


    1) MMM    : 1305



[*********************100%***********************]  1 of 1 completed


    2) AOS    : 1305



...

  499) ZBRA   : 1305



[*********************100%***********************]  1 of 1 completed


  500) ZBH    : 1305



[*********************100%***********************]  1 of 1 completed

  501) ZTS    : 1305



In [7]:
yf.download("CTAS", start="2020-01-01")

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['CTAS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-01 -> 2025-03-13)')


Price,Adj Close,Close,High,Low,Open,Volume
Ticker,CTAS,CTAS,CTAS,CTAS,CTAS,CTAS
Date,,,,,,


In [3]:
co_integrate = CoIntegrate()
df_coint = co_integrate.cal_cointegration(5)
df_coint

MMM-AOS
MMM-ABT
MMM-ABBV
MMM-ACN
MMM-ADBE
MMM-AMD
MMM-AES
MMM-AFL
MMM-A
MMM-APD
MMM-AKAM
MMM-ALB
MMM-ARE
MMM-ALGN
MMM-ALLE
MMM-LNT
MMM-ALL
MMM-GOOGL
MMM-GOOG
MMM-MO
MMM-AMZN
MMM-AMCR
MMM-AEE
MMM-AEP
MMM-AXP
MMM-AIG
MMM-AMT
MMM-AWK
MMM-AMP
MMM-AME
MMM-AMGN
MMM-APH
MMM-ADI
MMM-ANSS
MMM-AON
MMM-APA
MMM-APO
MMM-AAPL
MMM-AMAT
MMM-APTV
MMM-ACGL
MMM-ADM
MMM-ANET
MMM-AJG
MMM-AIZ
MMM-T
MMM-ATO
MMM-ADSK
MMM-ADP
MMM-AZO
MMM-AVB
MMM-AVY
MMM-AXON
MMM-BKR
MMM-BALL
MMM-BAC
MMM-BAX
MMM-BDX
MMM-BBY
MMM-TECH
MMM-BIIB
MMM-BLK
MMM-BX
MMM-BK
MMM-BA
MMM-BKNG
MMM-BWA
MMM-BSX
MMM-BMY
MMM-AVGO
MMM-BR
MMM-BRO
MMM-BLDR
MMM-BG
MMM-BXP
MMM-CHRW
MMM-CDNS
MMM-CZR
MMM-CPT
MMM-CPB
MMM-COF
MMM-CAH
MMM-KMX
MMM-CCL
MMM-CAT
MMM-CBOE
MMM-CBRE
MMM-CDW
MMM-CE
MMM-COR
MMM-CNC
MMM-CNP
MMM-CF
MMM-CRL
MMM-SCHW
MMM-CHTR
MMM-CVX
MMM-CMG
MMM-CB
MMM-CHD
MMM-CI
MMM-CINF
MMM-CSCO
MMM-C
MMM-CFG
MMM-CLX
MMM-CME
MMM-CMS
MMM-KO
MMM-CTSH
MMM-CL
MMM-CMCSA
MMM-CAG
MMM-COP
MMM-ED
MMM-STZ
MMM-COO
MMM-CPRT
MMM-GLW
MMM-CPAY
MMM-CTVA
MMM-CSGP
MMM

,ticker1,ticker2,pvalue,coint_t,percent_1,percent_5,percent_10,cointegrate
0,MMM,AOS,0.891070,-1.059020,-3.904858,-3.34082,-3.047704,0
1,MMM,ABT,0.530916,-1.995500,-3.904858,-3.34082,-3.047704,0
2,MMM,ABBV,0.927190,-0.854262,-3.904858,-3.34082,-3.047704,0
3,MMM,ACN,0.887593,-1.075379,-3.904858,-3.34082,-3.047704,0
4,MMM,ADBE,0.883556,-1.093841,-3.904858,-3.34082,-3.047704,0
...,...,...,...,...,...,...,...,...
119311,YUM,ZBH,0.753979,-1.518629,-3.904858,-3.34082,-3.047704,0
119312,YUM,ZTS,0.906357,-0.981192,-3.904858,-3.34082,-3.047704,0
119313,ZBRA,ZBH,0.603148,-1.853986,-3.904858,-3.34082,-3.047704,0
119314,ZBRA,ZTS,0.626247,-1.807142,-3.904858,-3.34082,-3.047704,0


In [4]:
df = pd.read_csv("./data/sentiment.csv")
df

,pub_date,ticker,publisher,period,title,content,prosusai,prosusai_title,prosusai_content,yiyanghkust,yiyanghkust_title,yiyanghkust_content,ziweichen,ziweichen_title,ziweichen_content,aventiq_ai,aventiq_ai_title,aventiq_ai_content
0,2025-03-26 10:24:00,AAPL,Insider Monkey,6 hours ago,Apple Inc. (AAPL) AI Woes Grow as Siri Upgrade...,We recently published a list of GTC Recap and ...,3,5,3,3,1,3,3,2,3,5,5,4
1,2025-03-26 10:24:00,AAPL,Bloomberg,6 hours ago,"Australia Unveils Tax Cuts, Rebates in Pre-Ele...",(Bloomberg) -- Australia’s center-left governm...,3,3,2,2,3,2,2,3,4,1,5,1
2,2025-03-26 10:24:00,AAPL,Motley Fool,6 hours ago,"The ""Magnificent Seven"" Stocks Are Selling Off...","The ""Magnificent Seven"" stocks have been incre...",2,3,1,1,4,1,1,3,1,1,1,1
3,2025-03-26 07:24:00,AAPL,Insider Monkey,9 hours ago,Apple Inc. (AAPL)’s iPhone 18 May Debut With A...,We recently published a list of 9 Trending AI ...,3,3,3,3,3,3,2,3,2,5,5,2
4,2025-03-26 04:24:00,AAPL,Investor's Business Daily,12 hours ago,Apple Joins AI Data Center Race After Siri Mess,Apple reportedly is in the process of placing ...,3,3,3,3,3,3,3,3,3,5,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,2025-02-24 16:32:00,BA,Motley Fool,30 days ago,NASA May Cancel Boeing's Big $82 Billion Space...,The aerospace giant's loss may be other compan...,3,1,4,1,1,1,1,1,2,1,1,1
3791,2025-02-26 16:32:00,BA,Motley Fool,last month,Here's What Boeing's Big News Means for Investors,There has been some interesting and mixed news...,1,3,1,2,5,1,3,3,3,1,3,1
3792,2025-02-26 16:32:00,BA,FreightWaves,last month,"Atlas Air, DHL terminate Polar Air Cargo joint...",Polar Air Cargo has ceased operating after Atl...,1,3,1,3,3,3,3,3,3,1,3,1
3793,2025-02-26 16:32:00,BA,Quartz,last month,"Elon Musk's DOGE overdrive, Trump bashes Boein...",The Trump administration is planning to lay of...,1,3,1,3,3,3,3,3,3,1,2,1


In [5]:
df_test = df.loc[:, ["prosusai", "yiyanghkust", "ziweichen", "aventiq_ai"]]
df_test

,prosusai,yiyanghkust,ziweichen,aventiq_ai
0,3,3,3,5
1,3,2,2,1
2,2,1,1,1
3,3,3,2,5
4,3,3,3,5
...,...,...,...,...
3790,3,1,1,1
3791,1,2,3,1
3792,1,3,3,1
3793,1,3,3,1


In [6]:
df_test["test"] = df_test.values.tolist()
df_test

,prosusai,yiyanghkust,ziweichen,aventiq_ai,test
0,3,3,3,5,"[3, 3, 3, 5]"
1,3,2,2,1,"[3, 2, 2, 1]"
2,2,1,1,1,"[2, 1, 1, 1]"
3,3,3,2,5,"[3, 3, 2, 5]"
4,3,3,3,5,"[3, 3, 3, 5]"
...,...,...,...,...,...
3790,3,1,1,1,"[3, 1, 1, 1]"
3791,1,2,3,1,"[1, 2, 3, 1]"
3792,1,3,3,1,"[1, 3, 3, 1]"
3793,1,3,3,1,"[1, 3, 3, 1]"


In [ ]:
cond = df_test["test"].map(lambda x: (5 in x) & (1 in x))
df.loc[cond, :].style.set_properties(
    subset=["title", "content"], **{"width": "500px", "white-space": "normal"}
)

In [31]:
ticker = "JNJ"
cols = ["pub_date", "ticker", "publisher", "period", "title", "content", "ziweichen"]
df_jnj = df.loc[df["ticker"] == ticker, cols].reset_index(drop=True)
df_jnj

,pub_date,ticker,publisher,period,title,content,ziweichen
0,2025-03-25 16:28:00,JNJ,Zacks,yesterday,Johnson & Johnson (JNJ) Stock Dips While Marke...,The latest trading day saw Johnson & Johnson (...,2
1,2025-03-25 16:28:00,JNJ,Zacks,yesterday,"ACIU Stock Crashes 18.3% in a Month: Buy, Sell...",AC Immune's shares decline 18.3% in a month. W...,1
2,2025-03-25 16:28:00,JNJ,Insider Monkey,yesterday,Is Johnson & Johnson (JNJ) The Best Healthcare...,We recently published a list of 13 Best Health...,3
3,2025-03-25 16:28:00,JNJ,Zacks,yesterday,Take the Zacks Approach to Beat the Markets: C...,"Last week, our time-tested methodologies serve...",5
4,2025-03-25 16:28:00,JNJ,Zacks,yesterday,Here's How to Play JNJ Stock as it Announces $...,Those who own JNJ stock may stay invested for ...,5
...,...,...,...,...,...,...,...
195,2025-01-26 16:28:00,JNJ,Pharma Voice,2 months ago,J&J is all in on tau-targeted Alzheimer’s cand...,Anti-tau candidates haven’t crossed the regula...,5
196,2025-01-26 16:28:00,JNJ,Zacks,2 months ago,Zacks Earnings Trends Highlights: Netflix and ...,Netflix and Johnson & Johnson have been highli...,3
197,2025-01-26 16:28:00,JNJ,GuruFocus.com,2 months ago,Johnson & Johnson (JNJ) Q4 2024 Earnings Call ...,"Despite a decline in quarterly net earnings, J...",5
198,2025-01-26 16:28:00,JNJ,Thomson Reuters StreetEvents,2 months ago,Q4 2024 Johnson & Johnson Earnings Call,Q4 2024 Johnson & Johnson Earnings Call,3


In [26]:
df_jnj["pub_date"] = pd.to_datetime(df_jnj["pub_date"])
df_jnj["date"] = df_jnj["pub_date"].dt.date

df_exclude = df_jnj.loc[df_jnj["ziweichen"] != 3, :]
df_exclude.loc[
    df_exclude["date"] == datetime.strptime("2025-01-24", "%Y-%m-%d").date(), :
]

,pub_date,ticker,publisher,period,title,content,ziweichen,date


In [32]:
gen_pa = GenPriceAction()
df_av = gen_pa.cal_mean_sentiment(df_jnj, "JNJ")
df_av

,av_rating,median_rating_excl
date,,
2025-01-26,4.200000,5.0
2025-02-24,3.000000,3.0
2025-02-25,4.000000,5.0
2025-02-26,3.084211,4.0
2025-02-27,2.000000,1.0
2025-02-28,3.000000,3.0
2025-03-01,3.000000,3.0
2025-03-02,3.666667,5.0
2025-03-03,3.000000,3.0


In [47]:
jnj = pd.read_csv("./data/results/2025-03-26/JNJ_PYPL.csv")
jnj

,date,ticker,day_name,is_holiday,av_rating,median_rating_excl,JNJ_close,PYPL_close,action
0,2025-01-26,JNJ,Sunday,False,4.200000,5.0,NaN,NaN,buy
1,2025-02-24,JNJ,Monday,False,3.000000,3.0,163.74,75.25,wait
2,2025-02-25,JNJ,Tuesday,False,4.000000,5.0,166.09,74.07,buy
3,2025-02-26,JNJ,Wednesday,False,3.084211,4.0,163.08,72.05,buy
4,2025-02-27,JNJ,Thursday,False,2.000000,1.0,163.73,70.73,sell
5,2025-02-28,JNJ,Friday,False,3.000000,3.0,165.02,71.05,wait
6,2025-03-01,JNJ,Saturday,False,3.000000,3.0,NaN,NaN,wait
7,2025-03-02,JNJ,Sunday,False,3.666667,5.0,NaN,NaN,buy
8,2025-03-03,JNJ,Monday,False,3.000000,3.0,167.28,69.75,wait
9,2025-03-04,JNJ,Tuesday,False,3.000000,3.0,165.42,67.48,wait


In [144]:
results = utils.load_csv("./data/results/2025-03-26/trade_results.csv")
results.loc[(results["ticker"] == "JNJ") & (results["coint_ticker"] == "PYPL"), :]

,ticker,coint_ticker,action,entry_date,entry_price,exit_date,exit_price,profit_loss,days_held,win
1062,JNJ,PYPL,buy,2025-02-25,74.07,2025-02-27,70.73,-3.34,2,0
1063,JNJ,PYPL,buy,2025-02-26,72.05,2025-02-27,70.73,-1.32,1,0
1064,JNJ,PYPL,buy,2025-03-06,68.08,2025-03-07,70.48,2.4,1,1
1065,JNJ,PYPL,buy,2025-03-11,67.98,2025-03-25,70.86,2.88,14,1
1066,JNJ,PYPL,buy,2025-03-12,68.62,2025-03-25,70.86,2.24,13,1
1067,JNJ,PYPL,buy,2025-03-14,68.95,2025-03-25,70.86,1.91,11,1
1068,JNJ,PYPL,buy,2025-03-19,70.0,2025-03-25,70.86,0.86,6,1
1069,JNJ,PYPL,buy,2025-03-21,70.19,2025-03-25,70.86,0.67,4,1


In [149]:
results.loc[results["ticker"] == "AAPL", :]

,ticker,coint_ticker,action,entry_date,entry_price,exit_date,exit_price,profit_loss,days_held,win


In [ ]:
a = results["entry_date"].min()
b = results["entry_date"].max()
(b - a).days

88

In [143]:
results.at[1067, "win"]

Decimal('1')

In [43]:
df_summary = results.groupby(by=["ticker", "coint_ticker"]).agg(
    {
        "entry_date": "min",
        "exit_date": "max",
        "entry_price": "sum",
        "profit_loss": ["min", "max", "sum"],
        "win": ["count", "sum"],
    }
)
df_summary

entry_date  exit_date entry_price profit_loss         \
                           min        max         sum         min    max   
ticker coint_ticker                                                        
AMZN   ABT          2025-03-14 2025-03-25      634.68       -1.61  -0.41   
       CAG          2025-03-14 2025-03-25      129.03       -0.55   0.35   
       CPAY         2025-03-14 2025-03-25     1742.63       -0.15  11.97   
       DLR          2025-03-14 2025-03-25      757.70       -4.07   1.04   
       DLTR         2025-03-14 2025-03-25      331.43        -2.6   2.55   
...                        ...        ...         ...         ...    ...   
XOM    ED           2025-02-25 2025-03-21      942.23       -1.05   5.16   
       EG           2025-02-25 2025-03-21     3170.56      -12.64   6.61   
       GPN          2025-02-25 2025-03-21      865.94       -0.33   5.12   
       MTCH         2025-02-25 2025-03-21      281.76       -0.87   0.41   
       NOC          2025-02-25 2025-03-21     4326.06       -3.39  15.62   

                             win      
                       sum count sum  
ticker coint_ticker                   
AMZN   ABT           -5.28     5   0  
       CAG           -0.98     5   1  
       CPAY          27.67     5   4  
       DLR           -6.12     5   3  
       DLTR          -0.05     5   3  
...                    ...   ...  ..  
XOM    ED            12.21     9   5  
       EG            -8.80     9   4  
       GPN           19.91     9   7  
       MTCH          -4.57     9   1  
       NOC           24.82     9   7  

[158 rows x 8 columns]

In [89]:
df_summary["percent_return"] = (
    df_summary[("profit_loss", "sum")] / df_summary[("entry_price", "sum")]
)
df_summary

entry_date  exit_date entry_price profit_loss         \
                           min        max         sum         min    max   
ticker coint_ticker                                                        
AMZN   ABT          2025-03-14 2025-03-25      634.68       -1.61  -0.41   
       CAG          2025-03-14 2025-03-25      129.03       -0.55   0.35   
       CPAY         2025-03-14 2025-03-25     1742.63       -0.15  11.97   
       DLR          2025-03-14 2025-03-25      757.70       -4.07   1.04   
       DLTR         2025-03-14 2025-03-25      331.43        -2.6   2.55   
...                        ...        ...         ...         ...    ...   
XOM    ED           2025-02-25 2025-03-21      942.23       -1.05   5.16   
       EG           2025-02-25 2025-03-21     3170.56      -12.64   6.61   
       GPN          2025-02-25 2025-03-21      865.94       -0.33   5.12   
       MTCH         2025-02-25 2025-03-21      281.76       -0.87   0.41   
       NOC          2025-02-25 2025-03-21     4326.06       -3.39  15.62   

                             win                          percent_return  
                       sum count sum                                      
ticker coint_ticker                                                       
AMZN   ABT           -5.28     5   0   -0.008319152958971450179618075251  
       CAG           -0.98     5   1   -0.007595132914826009455165465396  
       CPAY          27.67     5   4     0.01587829889305245519702977683  
       DLR           -6.12     5   3   -0.008077075359641018872904843606  
       DLTR          -0.05     5   3  -0.0001508614187007814621488700480  
...                    ...   ...  ..                                 ...  
XOM    ED            12.21     9   5     0.01295861944535835199473589251  
       EG            -8.80     9   4   -0.002775534921275736778360920468  
       GPN           19.91     9   7     0.02299235512853084509319352380  
       MTCH          -4.57     9   1    -0.01621947756956274843838727995  
       NOC           24.82     9   7    0.005737322182309075694743022519  

[158 rows x 9 columns]

In [132]:
annual_ret = (1 + df_summary["percent_return"]) ** (
    365 / df_summary["trading_period"]
) - 1
annual_ret = annual_ret.map(lambda num: num.quantize(Decimal("1.000000")))
df_summary["annualized_returns"] = annual_ret
df_summary

entry_date  exit_date entry_price profit_loss         \
                           min        max         sum         min    max   
ticker coint_ticker                                                        
AMZN   ABT          2025-03-14 2025-03-25      634.68       -1.61  -0.41   
       CAG          2025-03-14 2025-03-25      129.03       -0.55   0.35   
       CPAY         2025-03-14 2025-03-25     1742.63       -0.15  11.97   
       DLR          2025-03-14 2025-03-25      757.70       -4.07   1.04   
       DLTR         2025-03-14 2025-03-25      331.43        -2.6   2.55   
...                        ...        ...         ...         ...    ...   
XOM    ED           2025-02-25 2025-03-21      942.23       -1.05   5.16   
       EG           2025-02-25 2025-03-21     3170.56      -12.64   6.61   
       GPN          2025-02-25 2025-03-21      865.94       -0.33   5.12   
       MTCH         2025-02-25 2025-03-21      281.76       -0.87   0.41   
       NOC          2025-02-25 2025-03-21     4326.06       -3.39  15.62   

                             win                          percent_return  \
                       sum count sum                                       
ticker coint_ticker                                                        
AMZN   ABT           -5.28     5   0   -0.008319152958971450179618075251   
       CAG           -0.98     5   1   -0.007595132914826009455165465396   
       CPAY          27.67     5   4     0.01587829889305245519702977683   
       DLR           -6.12     5   3   -0.008077075359641018872904843606   
       DLTR          -0.05     5   3  -0.0001508614187007814621488700480   
...                    ...   ...  ..                                 ...   
XOM    ED            12.21     9   5     0.01295861944535835199473589251   
       EG            -8.80     9   4   -0.002775534921275736778360920468   
       GPN           19.91     9   7     0.02299235512853084509319352380   
       MTCH          -4.57     9   1    -0.01621947756956274843838727995   
       NOC           24.82     9   7    0.005737322182309075694743022519   

                    trading_period annualized_returns  
                                                       
ticker coint_ticker                                    
AMZN   ABT                      11          -0.242097  
       CAG                      11          -0.223518  
       CPAY                     11           0.686629  
       DLR                      11          -0.235933  
       DLTR                     11          -0.004994  
...                            ...                ...  
XOM    ED                       24           0.216299  
       EG                       24          -0.041389  
       GPN                      24           0.413001  
       MTCH                     24          -0.220181  
       NOC                      24           0.090903  

[158 rows x 11 columns]

In [145]:
df_summary.loc[("AMZN", "ABT"), "annualized_returns"]

Decimal('-0.242097')